# Quick Start

See the API for more detailed information, examples, formulas, and references for each function.

In [1]:
import numpy as np
import xarray as xr
import xskillscore as xs

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` replicates some verification data and `fct` some forecast (e.g. from a statistical or dynamical model).

In [2]:
obs = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.arange(4),
           np.arange(5),
       ],
       dims=["time", "lat", "lon"],
   )
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics.

### Correlation-Based 

* Pearson Correlation (`pearson_r`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation p value (`spearman_r_p_value`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Effective Sample Size (`effective_sample_size`)
* Coefficient of Determination (`r2`)

### Distance-Based

* Root Mean Squared Error (`rmse`)
* Mean Squared Error (`mse`)
* Mean Absolute Error (`mae`)
* Median Absolute Error (`median_absolute_error`)
* Symmetric Mean Absolute Percentage Error (`smape`)
* Mean Absolute Percentage Error (`mape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`.

In [3]:
r = xs.pearson_r(obs, fct, "time")
print(r)

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.39397283,  0.47339827,  0.97449907, -0.32762634, -0.34711691],
       [ 0.00581597, -0.26077424,  0.26538547, -0.92265954,  0.0443583 ],
       [ 0.8020954 ,  0.19361189, -0.68457501,  0.82433883,  0.92630021],
       [-0.40923515, -0.95851959,  0.90056895,  0.91102217,  0.55512613]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [4]:
p = xs.pearson_r_p_value(obs, fct, "time")
print(p)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.74220081, 0.6860543 , 0.14407891, 0.78750259, 0.7743214 ],
       [0.99629742, 0.83204436, 0.82900157, 0.2520218 , 0.97175136],
       [0.40743704, 0.87595949, 0.51997563, 0.38309427, 0.24594178],
       [0.73159114, 0.18400492, 0.28630051, 0.27058898, 0.62533997]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [5]:
r = xs.pearson_r(obs, fct, ["lat", "lon"])
print(r)

<xarray.DataArray (time: 3)>
array([-0.19752913,  0.05566859,  0.0228352 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


All deterministic metrics except for `pearson_r_eff_p_value`, `spearman_r_eff_p_value`, and `effective_sample_size` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [6]:
obs2 = xr.DataArray(
       np.random.rand(3, 180, 360),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.linspace(-89.5, 89.5, 180),
           np.linspace(-179.5, 179.5, 360),
       ],
       dims=["time", "lat", "lon"],
    )
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [7]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [8]:
r_weighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=weights)
print(r_weighted)

<xarray.DataArray (time: 3)>
array([-0.0034055 ,  0.00075357, -0.00193341])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [9]:
r_unweighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=None)
print(r_unweighted)

<xarray.DataArray (time: 3)>
array([-0.00640401,  0.00323411, -0.00396749])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


You can also pass the optional boolean kwarg `skipna=...`. If `True`, ignore any NaNs (pairwise) in `a` and `b` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [10]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

<xarray.DataArray (time: 3, lat: 4, lon: 5)>
array([[[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.70235727, 0.69535936, 0.49793784, 0.10555698, 0.46754343],
        [0.59742217, 0.17300017, 0.57625785, 0.49375849, 0.4606503 ]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.59032781, 0.22693078, 0.15173855, 0.0192703 , 0.66264062],
        [0.66304252, 0.44292115, 0.3082468 , 0.25250437, 0.3464455 ]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.33456252, 0.73798437, 0.17024018, 0.914595  , 0.63224106],
        [0.07446116, 0.05648255, 0.35007232, 0.85853172, 0.39081563]]])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00
  * lat      (lat) int64 

In [11]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=True)
print(mae_with_skipna)

<xarray.DataArray (time: 3)>
array([0.2367272 , 0.2279569 , 0.27741043])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [12]:
mae_without_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=False)
print(mae_without_skipna)

<xarray.DataArray (time: 3)>
array([nan, nan, nan])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics, mostly ported from `properscoring`.

* Brier Score (`brier_score`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Continuous Ranked Probability Score with a Gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)


We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations.

In [13]:
obs3 = xr.DataArray(
       np.random.rand(4, 5),
       coords=[np.arange(4), np.arange(5)],
       dims=["lat", "lon"]
   )
fct3 = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[np.arange(3), np.arange(4), np.arange(5)],
       dims=["member", "lat", "lon"],
   )

Continuous Ranked Probability Score with the ensemble distribution.

In [14]:
crps_ensemble = xs.crps_ensemble(obs3, fct3)
print(crps_ensemble)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.20395934, 0.39407703, 0.4856716 , 0.58768595, 0.10739938],
       [0.16574391, 0.15003847, 0.36352479, 0.50005538, 0.21454678],
       [0.12517198, 0.12875253, 0.2883122 , 0.04547452, 0.13513201],
       [0.95434475, 0.0752638 , 0.11462525, 0.1395265 , 0.40365519]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [15]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"))
print(crps_gaussian)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.18276574, 0.39187627, 0.46264277, 0.56615436, 0.11521784],
       [0.17745744, 0.13931713, 0.35176269, 0.49643345, 0.11965099],
       [0.13839007, 0.12520822, 0.29960524, 0.03518644, 0.10950513],
       [0.95223996, 0.03945527, 0.11285102, 0.1444094 , 0.40290523]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [16]:
from scipy.stats import norm
crps_quadrature = xs.crps_quadrature(obs3, norm)
print(crps_quadrature)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.27430635, 0.4230291 , 0.23581254, 0.5988753 , 0.435524  ],
       [0.52892805, 0.31554689, 0.36596961, 0.54600084, 0.29535135],
       [0.23382954, 0.23556691, 0.59196376, 0.40936447, 0.36435231],
       [0.59401922, 0.34839059, 0.33667646, 0.25570968, 0.54493577]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


We can also use a threshold Brier Score, to score hits over a certain threshold.

In [17]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.7)
print(threshold_brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.11111111, 1.        , 0.44444444, 1.        , 0.44444444],
       [0.44444444, 0.        , 0.        , 1.        , 0.44444444],
       [0.        , 0.        , 0.44444444, 0.44444444, 0.        ],
       [1.        , 0.11111111, 0.        , 0.11111111, 1.        ]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


## Contingency-Based

To work with contingency-based scoring, first instantiate a `Contingency` object by passing in your observations, forecast, and observation/forecast bin edges. See https://www.cawcr.gov.au/projects/verification/#Contingency_table for more information.

In [18]:
dichotomous_category_edges = np.array([0, 0.5, 1]) # "dichotomous" mean two-category
dichotomous_contingency = xs.Contingency(obs, fct,
                                        dichotomous_category_edges,
                                        dichotomous_category_edges,
                                        dim=['lat','lon'])
dichotomous_contingency_table = dichotomous_contingency.table

In [19]:
dichotomous_contingency_table

<xarray.DataArray 'histogram_observations_forecasts' (time: 3, observations_category: 2, forecasts_category: 2)>
array([[[3, 9],
        [4, 4]],

       [[7, 4],
        [4, 5]],

       [[2, 9],
        [4, 5]]])
Coordinates:
  * time                          (time) object 2000-01-01 00:00:00 ... 2000-...
    observations_category_bounds  (observations_category) <U10 '(0.0, 0.5]' '...
    forecasts_category_bounds     (forecasts_category) <U10 '(0.0, 0.5]' '(0....
  * observations_category         (observations_category) int64 1 2
  * forecasts_category            (forecasts_category) int64 1 2

Scores based on the constructed contingency table can be called via class methods. The available methods are:

* bias_score
* hit_rate
* false_alarm_ratio
* false_alarm_rate
* success_ratio
* threat_score
* equit_threat_score
* odds_ratio
* odds_ratio_skill_score
* accuracy
* heidke_score
* peirce_score
* gerrity_score

Below, we share a few examples of these in action.

In [20]:
dichotomous_contingency.bias_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([1.625     , 1.        , 1.55555556])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [21]:
dichotomous_contingency.hit_rate()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.5       , 0.55555556, 0.55555556])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [22]:
dichotomous_contingency.false_alarm_rate()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.75      , 0.36363636, 0.81818182])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [23]:
dichotomous_contingency.odds_ratio_skill_score()


<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.5       ,  0.37254902, -0.56521739])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

Now we can leverage multi-category edges to make use of some scores.

In [24]:
multi_category_edges = np.array([0, 0.25, 0.75, 1])
multicategory_contingency = xs.Contingency(obs, fct,
                                              multi_category_edges,
                                              multi_category_edges,
                                              dim=['lat','lon'])

In [25]:
multicategory_contingency.accuracy()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.5 , 0.35, 0.4 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [26]:
multicategory_contingency.heidke_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([ 0.03846154, -0.19266055,  0.03225806])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [27]:
multicategory_contingency.peirce_score()

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([ 0.04761905, -0.18918919,  0.03571429])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

In [28]:
multicategory_contingency.gerrity_score()

/Users/ribr5703/miniconda3/envs/xskillscore-dev/lib/python3.6/site-packages/xskillscore/core/contingency.py:698: RuntimeWarning: divide by zero encountered in true_divide
  np.sum(1.0 / a[..., 0:j], axis=-1)


<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.12380952, -0.06666667,  0.125     ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00

## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [29]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [30]:
ds.xs.pearson_r("obs_var", "fct_var", "time")

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.39397283,  0.47339827,  0.97449907, -0.32762634, -0.34711691],
       [ 0.00581597, -0.26077424,  0.26538547, -0.92265954,  0.0443583 ],
       [ 0.8020954 ,  0.19361189, -0.68457501,  0.82433883,  0.92630021],
       [-0.40923515, -0.95851959,  0.90056895,  0.91102217,  0.55512613]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4

You can also pass in a separate Dataset that contains your observations or forecast variable.

In [31]:
ds = ds.drop_vars("fct_var")
ds.xs.pearson_r("obs_var", fct, "time")

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.39397283,  0.47339827,  0.97449907, -0.32762634, -0.34711691],
       [ 0.00581597, -0.26077424,  0.26538547, -0.92265954,  0.0443583 ],
       [ 0.8020954 ,  0.19361189, -0.68457501,  0.82433883,  0.92630021],
       [-0.40923515, -0.95851959,  0.90056895,  0.91102217,  0.55512613]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4